In [1]:
from openni import openni2
import numpy as np
import cv2 as cv
import time
import route
import route_control
import socket

In [4]:
    cap.release()
    dev.close()

In [2]:
#仿射变换矩阵参数调试 取一张图片
fpoint= []
def getpos1(event, x, y, flags, param):
    if event==cv.EVENT_LBUTTONDOWN:
        print("pos is", [x,y])
        fpoint.append([x,y])
        
        
if __name__ == "__main__": 
 
    openni2.initialize("./Win64-Release/sdk/libs")
    dev = openni2.Device.open_any()
    print(dev.get_device_info())

    cap = cv.VideoCapture(1)
    cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)  # 设置分辨率
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
    ret,frame = cap.read() 
    time.sleep(3)
    if  frame is not None :
        cv.imshow("affine_console",frame)
        cv.setMouseCallback("affine_console", getpos1)
        cv.waitKey(0)
        cv.destroyAllWindows()
    else:
        print("not ready")
    
    

   
    



OniDeviceInfo(uri = b'\\\\?\\usb#vid_2bc5&pid_0403&mi_00#8&2372ea8c&0&0000#{c3b5f022-5a42-1980-1909-ea72095601b1}', vendor = b'Orbbec', name = b'Astra', usbVendorId = 11205, usbProductId = 1027)
pos is [286, 303]
pos is [437, 271]
pos is [510, 412]
pos is [322, 455]


In [22]:
ret

True

In [4]:
#求仿射变换矩阵参数M  
xyrange = []
def getpos2(event, x, y, flags, param):
    if event==cv.EVENT_LBUTTONDOWN:
        print("pos is", [x,y])
        xyrange.append([x,y])
def cvt_pos(pos, cvt_mat_t):
    u = pos[0]
    v = pos[1]
    x = (cvt_mat_t[0][0]*u+cvt_mat_t[0][1]*v+cvt_mat_t[0][2])/(cvt_mat_t[2][0]*u+cvt_mat_t[2][1]*v+cvt_mat_t[2][2])
    y = (cvt_mat_t[1][0]*u+cvt_mat_t[1][1]*v+cvt_mat_t[1][2])/(cvt_mat_t[2][0]*u+cvt_mat_t[2][1]*v+cvt_mat_t[2][2])
    return [int(x), int(y)]
#求变换矩阵M
xc,yc = 600,800
width,height = 175,175
windowsize = 1200

min_plane_r = width*71/80


pts_o = np.float32([fpoint[0],fpoint[1],fpoint[2],fpoint[3]])
pts_d = np.float32([[xc , yc],[xc + width, yc],[xc +width,yc + height],[xc,yc + height]])
M = cv.getPerspectiveTransform(pts_o,pts_d)
frame_affine = cv.warpPerspective(frame,M, (windowsize,windowsize))

cv.namedWindow("img_affine",cv.WINDOW_NORMAL)
cv.imshow('img_affine',frame_affine)
cv.setMouseCallback("img_affine", getpos2)
cv.waitKey(0)
cv.destroyAllWindows()#摧毁所有窗口

frame_cut = frame_affine[xyrange[0][1]:xyrange[1][1] , xyrange[0][0] :xyrange[1][0]]
height = xyrange[1][1] - xyrange[0][1]   #窗口高度
cv.imshow('img_affine',frame_cut)
cv.waitKey(0)
cv.destroyAllWindows()#摧毁所有窗口

pos is [120, 21]
pos is [1183, 1092]


In [5]:

        

red_hsv_low =  np.array([140 ,15 ,150])
red_hsv_high = np.array([180 ,150 ,255])
orange_hsv_low =  np.array([1 ,100 ,100])
orange_hsv_high = np.array([15 ,200 ,255])
blue_hsv_low =  np.array([85 ,100 ,55])
blue_hsv_high = np.array([105 ,255 ,255])
white_hsv_low =  np.array([0 ,0 ,221])
white_hsv_high = np.array([180 ,30 ,255])
black_hsv_low =  np.array([90 ,130 ,46])
black_hsv_high = np.array([100 ,150,120])
green_hsv_low =  np.array([45 ,50 ,150])
green_hsv_high = np.array([70 ,155,255])
yellow_hsv_low =  np.array([22 ,46 ,46])
yellow_hsv_high = np.array([28 ,155,255])
purple_hsv_low =  np.array([115 ,40 ,146])
purple_hsv_high = np.array([155,255,255])


def colordetect(img, hsv_low , hsv_high, minsize, blur, kernel_dilate,kernel_erode):
    posi =[]
    blur0 = cv.medianBlur(img,blur)

    hsv = cv.cvtColor(blur0, cv.COLOR_BGR2HSV)

    mask = cv.inRange(hsv, hsv_low, hsv_high)
    
    kernel_d = cv.getStructuringElement(cv.MORPH_RECT, (kernel_dilate, kernel_dilate))
    kernel_e = cv.getStructuringElement(cv.MORPH_RECT, (kernel_erode, kernel_erode))

    dilated = cv.dilate(mask, kernel_d)
    eroded = cv.erode(dilated, kernel_e)
    contours, hierarchy = cv.findContours(eroded, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    box = []

    for i in range(0, len(contours)):    
        if cv.contourArea(contours[i])>minsize:
            rect = cv.minAreaRect(contours[i])#得出轮廓最小外接矩形左上角坐标及长、宽信息
            box = cv.boxPoints(rect) # 获取最小外接矩形的4个顶点坐标(ps: cv2.boxPoints(rect) for OpenCV 3.x)
            box = np.int0(box)   # 画出来
            posi  = np.average(box,axis = 0)
            cv.drawContours(blur0, [box], 0, (255, 0, 0), 1)
            break

    return posi, mask, blur0,eroded , dilated,box

def drawroute(img, rc_car , rc_plane ,  theta_car1,theta_car2 , theta_plane1, theta_plane2 , radius_car,radius_plane, window_height):
    radius_car = int(radius_car)

    cv.ellipse(img,(int(rc_car[0]),window_height-int(rc_car[1])),(radius_car,radius_car),0,theta_car1,theta_car2,(255,255,0),2)  
    cv.ellipse(img,(int(rc_plane[0]),window_height-int(rc_plane[1])),(radius_plane,radius_plane),0,theta_plane1,theta_plane2,(125,255,255),2)
    
    
    

#cv.ellipse(image0,(400,400),(100,100),0,30,60,(255,255,0),2)   


In [14]:




#position , mask_img , blur_img ,eroded,dilated,box = colordetect(frame_cut,  black_hsv_low, black_hsv_high ,300, 3 ,20,20)
#position , mask_img , blur_img ,eroded,dilated,box = colordetect(frame_cut,  blue_hsv_low, blue_hsv_high ,15, 1 ,3,3)
#position , mask_img , blur_img ,eroded, dilated, box = colordetect(frame_cut,  yellow_hsv_low, yellow_hsv_high ,50,  1 ,5,5)
#position , mask_img , blur_img ,eroded, dilated, box = colordetect(frame_cut,  red_hsv_low, red_hsv_high ,20,  1 ,3,1)
#position , mask_img , blur_img ,eroded, dilated, box = colordetect(frame_cut, green_hsv_low, green_hsv_high ,30, 1 ,1,1)
position , mask_img , blur_img ,eroded, dilated, box = colordetect(frame_cut,  orange_hsv_low, orange_hsv_high ,30,  1 ,1,1)
#position , mask_img , blur_img ,eroded, dilated, box = colordetect(frame_cut,  purple_hsv_low, purple_hsv_high ,30,  1 ,3,3)
print(position)

hsv = cv.cvtColor(frame_cut, cv.COLOR_BGR2HSV)
def getposHsv(event, x, y, flags, param):
    if event==cv.EVENT_LBUTTONDOWN:
        print("HSV is", hsv[y, x])
        
cv.imshow('img_frame',frame_cut)

cv.setMouseCallback("img_frame", getposHsv)
cv.imshow("imgblur0", blur_img)
cv.imshow("mask", mask_img)
cv.imshow("erode", eroded)
cv.imshow("dilated", dilated)
cv.waitKey(0)
cv.destroyAllWindows()

[]


In [25]:
position

array([705.  , 895.25])

In [6]:
# image0 = cv.imread(r'C:\Users\longyuhan\Desktop\plane.png')
# image0 = cv.resize(image0, (640,480),interpolation=cv.INTER_CUBIC)
def calculate(rect , head):
    center  = np.average(rect,axis = 0)
    obj_edge_mid= []
    for i in range(4):
        obj_edge_mid.append(( rect[i] + rect[(i+1)%4])/2)

    len_head2edgemid =[]
    for i in obj_edge_mid:
        len_head2edgemid.append(sum((i-head) ** 2))
    
    ind_cardirct = len_head2edgemid.index(min(len_head2edgemid))
    car_dirction  =  obj_edge_mid[ind_cardirct]-center

    return center, car_dirction
 
def arrow(image, center,car_dirction,l,f,tipLen):
    center = np.array(center)
    car_dirction = np.array(car_dirction)
    end = np.array(car_dirction*l+center)
    cv.arrowedLine(image,
                pt1=np.int0(center),
                pt2=np.int0(end),
                color=f,
                thickness=2,
                line_type=cv.LINE_8,
                shift=0,
                tipLength=tipLen)

#cen,dirc = calculate(box1,position)
#arrow(image0,cen,dirc)
# cv.imshow('img_frame',image0)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [28]:
#通信socket调试
HOST = '192.168.137.226'  #树莓派ip 
PORT = 2218 
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))
sock.send(bytes(str("hello"),encoding='utf-8'));

In [26]:
pos_blue[1]

639.5

In [7]:
HOST = '192.168.137.226'  #树莓派ip 
PORT = 2218 
car_v_pwm = 420
car_v_stop = 400
car_steer = 360
car_v_reverse = 350   #倒车360动  355拉动
min_plane_r = 200
font = cv.FONT_HERSHEY_SIMPLEX

slide_distance = 120
if __name__ == "__main__": 
    velocity = 0
    v_stop = 0
    t1 = 0
    R = 200
    pos_stop = np.array([0,0])
    x1 = np.array([0,0])
    gui = False
    slide = False
    catch = False
    reverse = False
    goback = False
    align = False
    step = 180
    stop = False
    a  = 0
    angle = 90    #0放下
    arr = ["forward","left","right","slide"]
    arr_back = ["back","back left","back right"]
    car_p = car_v_pwm
    height = xyrange[1][1] - xyrange[0][1]   #窗口高度
    length = xyrange[1][0] - xyrange[0][0]   #窗口宽度
    count = 0
    control = False
    openni2.initialize("./Win64-Release/sdk/libs")
    dev = openni2.Device.open_any()
    print(dev.get_device_info())

    cap = cv.VideoCapture(1)
    cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)  # 设置分辨率
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
    #cap.set(cv.CAP_PROP_FPS,30)
    #cv.SetCaptureProperty(cap, cv.CV_CAP_PROP_FPS,30)
    router = False
    while True:
        
        ret,frame = cap.read()     #读取摄像头图像
        #print("fps:",cap.get(cv.CAP_PROP_FPS))
        while frame is  None:
            print("not ready")
            time.sleep(3)
            
            ret,frame = cap.read()
            
            break
        else:
            frame_affine = cv.warpPerspective(frame,M, (windowsize,windowsize))    #透视变换
            frame_cut = frame_affine[xyrange[0][1]:xyrange[1][1] , xyrange[0][0] :xyrange[1][0]]
            
            pos_red , mask_img , blur_red ,eroded,dilated,box_red = colordetect(frame_cut,  red_hsv_low, red_hsv_high, 20,  1 ,1,1) #颜色识别
            pos_green , mask_img , blur_green ,eroded, dilated, box_black = colordetect(frame_cut, green_hsv_low, green_hsv_high ,20, 1 ,3,1)
           
            pos_blue , mask_img , blur_blue ,eroded, dilated, box_black = colordetect(frame_cut, blue_hsv_low, blue_hsv_high ,15, 1 ,3,3)
            pos_orange , mask_img , blur_orange ,eroded, dilated, box_black = colordetect(frame_cut, orange_hsv_low, orange_hsv_high ,30,  1 ,1,1)
           
        
          #识别目标，计算位置
                                    
            if len(pos_red)!= 0 and len(pos_green)!= 0 and len(pos_blue)!= 0 and len(pos_orange)!= 0:  
                dirc_plane = pos_green-pos_red
                dirc_plane = dirc_plane/np.linalg.norm(dirc_plane)
                
                #标定校准
                pos_blue[1] = pos_blue[1] + (height-pos_blue[1])*65/height
                pos_orange[1] = pos_orange[1] + (height-pos_orange[1])*65/height
                
                if reverse == False:
                    dirc_car = pos_orange-pos_blue 
                    pos_car  = pos_blue
                else: 
                        dirc_car = pos_blue-pos_orange
                        pos_car  = pos_orange
                dirc_car =  dirc_car/np.linalg.norm(dirc_car)
                pos_plane  = pos_green      
                
                velocity = np.linalg.norm(pos_car - x1)/(count - t1)
                
                
               # arrow(frame_cut, x1, pos_car , np.linalg.norm(pos_car - x1),(125, 125, 255),0.2)
                
                x1 = pos_car
                t1 = count
                               
           
                arrow(frame_cut,pos_plane,dirc_plane,50,(255, 255, 0),0.2)                 #画出箭头
                arrow(frame_cut,pos_car,dirc_car,50,(125, 255, 255),0.2)
                cv.putText(frame_cut,"AGV ("+ str(pos_car[0]) + "," + str(height - pos_car[1]) + ")", np.int0(pos_car), font, 0.5, (0, 0,0), 1) #字体大小，字体颜色，字体粗细
                cv.putText(frame_cut,"UAV ("+ str(pos_plane[0]) + "," + str(height - pos_plane[1]) + ")", np.int0(pos_plane), font, 0.5, (0, 0,0), 1)
                cv.putText(frame_cut,"velocity: "+ str(velocity), [length-250,50], font, 0.5, (0, 0,0), 1)
                cv.putText(frame_cut,"stop_v: "+ str(v_stop), [length-250,100], font, 0.5, (0, 0,0), 1)
                cv.putText(frame_cut,"slide_distance: "+ str(np.linalg.norm(pos_stop-pos_car)), [length-250,150], font, 0.5, (0, 0,0), 1)
                cv.putText(frame_cut,"steer: "+ str(car_steer), [length-250,200], font, 0.5, (0, 0,0), 1)
                
                if reverse == False:
                    cv.putText(frame_cut, arr[a],np.int0(pos_car+[0,-30]) ,font,0.8,(240,128,128), 2)  
                elif slide == False:
                    cv.putText(frame_cut, arr_back[a],np.int0(pos_car+[0,-30]) ,font,0.8,(240,128,128), 2)  
                else :
                    cv.putText(frame_cut, arr[3],np.int0(pos_car+[0,-30]) ,font,0.8,(240,128,128), 2)  

        #可视化  
            
            cv.putText(frame_cut, str(height),[10,20],font,0.6,(0,128,0),2)                    #坐标系可视化
            cv.putText(frame_cut, str(length),[length-50,height-10],font,0.6,(0,128,0),2)
            arrow(frame_cut,[2,height-2],[1,0],length,(0,128,0),0.02)
            arrow(frame_cut,[0,height-2],[0,-1],height,(0,128,0),0.02)
            
            
            
            if router is True:
                cv.line(frame_cut,np.int0(pos_plane_route), np.int0(pos_slide),(218,165,32), 2 )       #画出滑行轨道       
                drawroute(frame_cut, rc_car, rc_plane , theta_car1,theta_car2 , theta_plane1, theta_plane2 , radius_car, np.int0(min_plane_r),height)
       
            
        
    
            if  router == True :             # 0.1*n 
                plane_position = [pos_plane[0],height-pos_plane[1]]
                car_position = [pos_car[0],height-pos_car[1]]
                car_dir = [dirc_car[0],-dirc_car[1]]
#                 print("小车位置,方向",car_position , car_dir)
#                 print("切圆、接驳圆位置，",rc_car, rc_plane,dir_c,dir_p)
#                 print("切圆、接驳圆半径" ,radius_car,min_plane_r)
             
                    
                
                a=route_control.which_way_to_go(reverse, step ,pos_c, car_position ,car_dir,np.pi/6, rc_plane, rc_car, min_plane_r ,radius_car,dir_c,dir_p,theta)
               
                    
     
                
                
                
                
                if control == True:
                    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    sock.connect((HOST, PORT))  
                    #print("velocity",velocity)
                    R = velocity*velocity/30       
                    print(count)
                    
                    #print("R",R)
                    if np.linalg.norm(pos_car - (pos_plane + dirc_plane*75))  < 120  and slide == False and gui == False and catch == False:
                        car_p = car_v_stop
                        stop_point = count
                        #angle = 90
                        car_steer = 360
                        v_stop  = velocity
                        print("v_stop" ,v_stop)
                        pos_stop = pos_car
                        slide = True
                        print("slide!")    #停车滑行
                        
                    #if slide == True:
#                         if np.linalg.norm(pos_car - (pos_plane + dirc_plane*60) ) < 30 and stop == False:
#                             car_p = car_v_reverse
#                             stop_time = count
#                             stop = True
#                             print("stop")
#                         if stop == True :
#                             if count> stop_time +3:
#                                 car_p = car_v_stop
                                
                                
                    if slide == True :          
                        if  count >  stop_point + 10  :
                            catch = True
                            angle = 0
                            print("catch!")
                            catch_point = count
                            slide = False
#                             if np.linalg.norm(pos_car - (pos_plane + dirc_plane*60) ) < 30  and goback == False:
#                                 gui = True
#                                 print("gui")
#                                 slide = False
#                                 print("slide_distance",np.linalg.norm(pos_stop-pos_car))
#                             else:
#                                 car_p = 355
#                                 goback = True
#                                 angle = 0
#                                 print("goback!")
#                                 back_point = count
#                                 slide = False
#                     if gui == True:
#                         if   np.linalg.norm(pos_car - (pos_plane + dirc_plane*60) ) < 30 and count%6 == 5:
#                             catch = True
#                             gui = False
#                             angle = 0
#                             print("catch!")
#                             catch = True
#                             catch_point = count
#                         else: 
#                             if count % 6 <= 0:
#                                 print("移动",count)
#                                 car_p = car_v_pwm
#                             else :   car_p = 415

                    if   catch == True :
                        if   count > catch_point + 10 : 
                            if route.get_route(pos_p,dirc_p, np.array([length/2,0]),np.array([0,1]),min_plane_r) != 0:
                                rc_car , rc_plane , theta_car1,theta_car2 , theta_plane1, theta_plane2 , radius_car,dir_c,dir_p,theta  = route.get_route(pos_p,dirc_p, np.array([length/2,0]),np.array([0,1]),min_plane_r)
                            car_p = car_v_reverse
                            reverse = True
                            print("reverse!")
                            
#                     if slide == False and reverse == False and catch == False and gui == False:
#                         if count % 10 <= 6:               #调速使匀速
#                             car_p = car_v_pwm
#                         else:
#                             car_p = 415
                    #倒车回退
#                     if goback == True:
#                             a = 0
#                             slide = False
#                             if count  > back_point + 16:
#                                 car_p = car_v_pwm 
#                                 align = True
#                                 goback = False
                    
                    
                    if reverse == True:
                        if a == 1:
                            a = 2
                        elif a == 2:
                            a = 1
                            
                        if car_position[1] < 10:                  #停车到车库
                            car_p = car_v_stop
                    
                    
                    
                    
                    
                    
                    # 出界控制
#                     if (car_position[0] < 100 and car_dir[0]<-0.7 ) or (car_position[0] > length-100 )or car
                        
            
            
            
                    
                    if a == 0:                #前进                                            # 400左  320右
                        car_steer = 360
#                         print("发送前进")
                    elif a == 1:  #左转
                        if car_steer <= 420:
                            #car_steer = car_steer + 30
                            car_steer = 420
#                         print("发送左转",car_steer)
                    elif a == 2:   #右转 
                        if car_steer >= 300:
                            #car_steer = car_steer - 30
                            car_steer = 300
#                         print("发送右转",car_steer)
                    
    
                 #精准对准？
                    if   (slide == True or gui == True)  and catch == False and goback == False :
                        car_position = np.array(car_position)
                        plane_position = np.array(plane_position)
                        dirc_p = [-dirc_plane[0], dirc_plane[1]]
                        d = np.linalg.norm(car_position-plane_position)
                        dirction1 =  (plane_position-car_position)/d
                        sin1 = dirction1[0]*dirc_p[1] - dirction1[1]*dirc_p[0]
                        l = d * abs(sin1)
                        if l > 60:
#                             if sin1 >0 : car_steer = 300
#                             else : car_steer = 420
                            car_steer  = 360 - 80*sin1
                            print("L>60,",car_steer)
                        else:
                             dirc_p = [-dirc_plane[0], dirc_plane[1]]
                             sin = car_dir[0]*dirc_p[1] - car_dir[1]*dirc_p[0]
                             car_steer  = 360 + 80*sin
                             print("L<60,",car_steer)
                                


                    sock.send(bytes(str(car_p)+","+str(car_steer)+","+str(angle),encoding='utf-8'));
                    
                    if car_p == car_v_pwm:
                        time.sleep(1)
                        car_p = car_v_pwm - 1
        
           # cv.imshow("rgb_img",frame)
            cv.imshow("frame_green",blur_green)
            cv.imshow("frame_red",blur_red)
            cv.imshow("frame_blue",blur_blue)
            cv.imshow("frame_orange",blur_orange)
            cv.namedWindow('frame', cv.WINDOW_NORMAL)
            cv.imshow("frame",frame_cut)
            time.sleep(0.1)
            count= count+1
            key = cv.waitKey(1)
                        
            #按键控制 q退出 c规划路径 s开始通信 p车制动
            
            if int(key) == ord('q'):
                print("quit")
                cv.destroyAllWindows()
                sock.close()
                break
            if int(key) == ord('c') :
                pos_plane_route = pos_plane
                pos_slide = pos_plane_route + dirc_plane*slide_distance
                pos_p = np.array([pos_slide[0],height-pos_slide[1]])
                pos_c = np.array([pos_car[0],height-pos_car[1]])  
                dirc_p = np.array([dirc_plane[0], -dirc_plane[1]])
                dirc_c = np.array([dirc_car[0], -dirc_car[1]])
                
                if route.get_route(pos_p,dirc_p,pos_c,dirc_c,min_plane_r) != 0:
                    rc_car , rc_plane , theta_car1,theta_car2 , theta_plane1, theta_plane2 , radius_car,dir_c,dir_p,theta  = route.get_route(pos_p,dirc_p,pos_c,dirc_c,min_plane_r)
                    #rc_car , rc_plane , theta_car1,theta_car2 , theta_plane1, theta_plane2 , radius_car,dir_c,dir_p,theta  = route.get_route(pos_p,dirc_p, np.array([length/2,0]),np.array([0,1]),min_plane_r)

                    router = True
                    #print("路径输入参数：",pos_p,dirc_p,pos_c,dirc_c,min_plane_r)
                    print("plan a route")
                
                cv.destroyAllWindows()
                
            if int(key) == ord('s') :
                control = not control
                
                cv.destroyAllWindows()
            if int(key) == ord('p') :
                if car_p == car_v_stop:
                    car_p = car_v_pwm
                else :
                    car_p = car_v_stop
                    v_stop  = velocity
                    pos_stop = pos_car
                cv.destroyAllWindows()
        
#     cap.release()
#     dev.close()
#     cv.destroyAllWindows()

OniDeviceInfo(uri = b'\\\\?\\usb#vid_2bc5&pid_0403&mi_00#8&2372ea8c&0&0000#{c3b5f022-5a42-1980-1909-ea72095601b1}', vendor = b'Orbbec', name = b'Astra', usbVendorId = 11205, usbProductId = 1027)
looping
无人机的位置是[374.49097061 520.63963236]
无人机的方向向量是[-0.94174191 -0.3363364 ]
[[2, 378.811050693692], [3, 12.304360947285128]]
len(roots)=2
best_index=1
dist_list=[478.01682095  76.16646073]
最短路径的半径和方向是12.304360947285128,3
圆心角为[67.84266624297834, 64.37601139346185]
行驶时间为[14.569321862340683, 224.7146716234584]
looping
无人机的位置是[374.49097061 520.63963236]
无人机的方向向量是[-0.94174191 -0.3363364 ]
[[2, 378.811050693692], [3, 12.304360947285128]]
len(roots)=2
best_index=1
dist_list=[478.01682095  76.16646073]
最短路径的半径和方向是12.304360947285128,3
圆心角为[67.84266624297834, 64.37601139346185]
行驶时间为[14.569321862340683, 224.7146716234584]
plan a route


D:\anaconda\envs\orbbec\lib\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


127
128
129
130
v_stop 30.95933058785313
slide!
L>60, 319.377362961205
131
L<60, 415.80802781667444
132
L<60, 421.2833495147455
133
L<60, 421.17068968165125
134
L<60, 409.887238732309
135
L<60, 419.3474873535532
136
L<60, 414.85881230343904
137
L<60, 413.2155344379837
138
L<60, 416.1166792653855
139
L<60, 415.450105221124
140
L<60, 415.59083434485433
141
catch!
142
143
144
145
146
147
148
149
150
151
152
looping
无人机的位置是[374.49097061 520.63963236]
无人机的方向向量是[0.9329231300425794, 0.3600755940515219]
[[0, 597.4234949634293], [3, 342.66321245563944]]


TypeError: bad operand type for unary -: 'list'

In [25]:
print("路径输入参数：",pos_p,dirc_p,pos_c,dirc_c,min_plane_r)

路径输入参数： [213.71355174 457.52110534] [ 0.50213552 -0.86478895] [514.5 335.5] [-0.97341717 -0.22903933] 177.5


In [47]:
sin

-0.4937336812232105

In [23]:
dirction1

array([0.93641865, 0.35088474])

In [49]:
R

0.03334727980741498

In [66]:

def getpost(event, x, y, flags, param):
    if event==cv.EVENT_LBUTTONDOWN:
        print("pos is", [x,y])
        p.append([x,y])

image0 = cv.imread(r'C:\Users\longyuhan\Desktop\pictures\plane.png')
image0 = cv.resize(image0, (640, 680))
radius_plane = 250

p = []


cv.imshow("df",image0)
cv.setMouseCallback("df", getpost)
cv.waitKey(0)
cv.destroyAllWindows()

p = np.array(p)
pos_plane = p[0]
dirc_plane = p[1] - p[0]
dirc_plane = dirc_plane/np.linalg.norm(dirc_plane)
pos_car = p[2]
dirc_car = p[3] - p[2]
dirc_car = dirc_car/np.linalg.norm(dirc_car)

arrow(image0,pos_plane ,dirc_plane ,50,(0, 125, 0),0.2)
arrow(image0,pos_car, dirc_car, 50,(155, 155, 0),0.2)

pos_plane[1] = 680 - pos_plane[1] 
dirc_plane[1] = -dirc_plane[1]
pos_car[1] = 680 - pos_car[1] 
dirc_car[1] = -dirc_car[1]

rc_car , rc_plane , theta_car1,theta_car2 ,theta_plane1, theta_plane2 ,radius_car,dir_c,dir_p,theta  = route.get_route(pos_plane,dirc_plane,pos_car,dirc_car,radius_plane)
print("路径的输入：飞机、小车位置、方向",pos_plane,dirc_plane,pos_car,dirc_car)
print("路径参数",dir_c,dir_p)
drawroute(image0, rc_car, rc_plane , theta_car1,theta_car2 , theta_plane1, theta_plane2 , radius_car,radius_plane,680)
print("rc_car",rc_car)
print("rc_plane",rc_plane)
print("起点坐标",pos_car)
print("飞机坐标",pos_plane)
img = image0
p = []
while True:



    cv.imshow("df",img)
    cv.setMouseCallback("df", getpost)
    key = cv.waitKey(1)
    if int(key) == ord('c') :
        img = image0
        test_x = p[0][0]
        test_y = 680- p[0][1]
        print("小车坐标",test_x,test_y)
        p = np.array(p)

        test_dir = np.array([p[1][0]-p[0][0],p[0][1]-p[1][1]])
        test_dir = test_dir/np.linalg.norm(test_dir)
        print("小车方向",test_dir)

        arrow(img,p[0],np.array([test_dir[0],-test_dir[1]]),50,(255, 255, 0),0.2)

        a=route_control.which_way_to_go(False,150,pos_car,[test_x,test_y],test_dir,np.pi/6, rc_plane ,rc_car,radius_plane ,radius_car,dir_c,dir_p,theta)
        print("路径控制输入参数",rc_plane,rc_car,radius_plane,radius_car,dir_c,dir_p)
        print("选择方向",a)
        p = []
        cv.destroyAllWindows()

    if int(key) == ord('q'):
        print("quit")
        cv.destroyAllWindows()

        break



        


IndexError: index 0 is out of bounds for axis 0 with size 0

In [9]:
pos_car

array([173, 140])

In [37]:
def getpostc(event, x, y, flags, param):
    if event==cv.EVENT_LBUTTONDOWN:
        print("pos is", [x,y])
        
image1 = cv.imread(r'C:\Users\longyuhan\Desktop\pictures\plane.png')
image1 = cv.resize(image1, (640, 680))
rc_car , rc_plane , theta_car1,theta_car2 ,theta_plane1, theta_plane2 ,radius_car,dir_c,dir_p  = route.get_route([213.71355174,457.52110534],np.array([ 0.50213552,-0.86478895]),[514.5, 335.5],np.array([-0.97341717 ,-0.22903933]) ,177.5)
print("路径参数",rc_car , rc_plane,dir_c,dir_p)
drawroute(image1, rc_car, rc_plane , theta_car1,theta_car2 , theta_plane1, theta_plane2 , radius_car,radius_plane,680)

cv.imshow("df",image1)
cv.setMouseCallback("df", getpostc)
cv.waitKey(0)
cv.destroyAllWindows()


looping
无人机的位置是[213.71355174, 457.52110534]
无人机的方向向量是[ 0.50213552 -0.86478895]
[[2, 281.4516907802152], [3, 280.5648551469498]]
len(roots)=2
best_index=0
dist_list=[110.23461691 465.15435365]
最短路径的半径和方向是281.4516907802152,2
圆心角为[66.06082776771034, 7.038306333782273]
行驶时间为[324.5078752321948, 21.804387423703723]
路径参数 [450.03649332 609.46990833] [367.21359037 546.65016014] 0 0


In [8]:
import route_control
import imp
imp.reload(route_control)

<module 'route_control' from 'C:\\Users\\longyuhan\\Desktop\\orbbec\\route_control.py'>

In [31]:
import route
import imp
imp.reload(route)

<module 'route' from 'C:\\Users\\longyuhan\\Desktop\\orbbec\\route.py'>

In [23]:
def getpost(event, x, y, flags, param):
    if event==cv.EVENT_LBUTTONDOWN:
        print("pos is", [x,y])
        p.append([x,y])
p = []
image2 = cv.imread(r'C:\Users\longyuhan\Desktop\pictures\plane.png')
image2 = cv.resize(image2, (640, 680))

cv.circle(image2,(507, 680-523),324,(0,0,0),2)
cv.circle(image2,(45, 680-326),177,(0,125,0),2)



print("选择方向",a)
cv.imshow("df",image2)
cv.setMouseCallback("df", getpost)
cv.waitKey(0)
cv.destroyAllWindows()

test_x = p[0][0]
test_y = 680- p[0][1]
print("小车坐标",test_x,test_y)


test_dir = np.array([p[1][0]-p[0][0],p[0][1]-p[1][1]])
test_dir = test_dir/np.linalg.norm(test_dir)
print("小车方向",test_dir)

#a=route_control.which_way_to_go([test_x,test_y],test_dir,np.pi/6,[367.21358976,546.65015968] ,[450.03649241, 609.46990728],177 ,324,0,1)
a=route_control.which_way_to_go([345,150],[-0.98,0.149],np.pi/6,[507.07369566,523.25282617], [45.03479241,326.17736814],177 ,324,0,1)
#a=route_control.which_way_to_go(car_position ,car_dir,np.pi/6,rc_car, rc_plane, min_plane_r ,radius_car,dir_c,dir_p)

print(a)

选择方向 2
pos is [414, 491]
pos is [346, 493]
小车坐标 414 189
小车方向 [-0.99956775 -0.02939905]
当前逼近的是切圆
2


In [1]:
b = 1
a = b
b = b+2
print(a)

1


In [22]:
image0 = cv.imread(r'C:\Users\longyuhan\Desktop\pictures\plane.png')
drone_pos=np.array([np.random.rand(1)[0]*180+200,np.random.rand(1)[0]*180+200])
theta=np.random.rand(1)[0]*np.pi*2
drone_dir=np.array([np.cos(theta),np.sin(theta)])

#从场端获取车头坐标与方位
AGV_pos=[350,30  ]
AGV_dir=[0,1]
def drawroute(img, rc_car , rc_plane ,  theta_car1,theta_car2 , theta_plane1, theta_plane2 , radius_car,radius_plane, window_height):
    radius_car = int(radius_car)

    cv.ellipse(img,(int(rc_car[0]),window_height-int(rc_car[1])),(radius_car,radius_car),0,theta_car1,theta_car2,(255,255,0),2)  
    cv.ellipse(img,(int(rc_plane[0]),window_height-int(rc_plane[1])),(radius_plane,radius_plane),0,theta_plane1,theta_plane2,(125,255,255),2)
    
    
    

#cv.ellipse(image0,(400,400),(100,100),0,30,60,(255,255,0),2)   
radius_plane = 71
rc_car , rc_plane ,  theta_car1,theta_car2 , theta_plane1, theta_plane2 , radius_car,dir_c,dir_p  = route.get_route(drone_pos,drone_dir,AGV_pos,AGV_dir)
drawroute(image0, rc_car , rc_plane   ,  theta_car1,theta_car2 , theta_plane1, theta_plane2 , radius_car,radius_plane,750)

cv.imshow('img_frame',image0)
cv.waitKey(0)
cv.destroyAllWindows()

TypeError: get_route() missing 1 required positional argument: 'r_min'

In [7]:
dir_p

1

In [10]:
image0 = cv.imread(r'C:\Users\longyuhan\Desktop\pictures\plane.jpg')
while True :
    cv.imshow('img_frame',image0)
    key = cv.waitKey(1)
    if int(key) == ord('q'):
            print("q")
            cv.destroyAllWindows()
            break
    if int(key) == ord('m'):
            print("m")
            cv.destroyAllWindows()
        


q


In [12]:
image0.shape[0]

1856

In [15]:
arctan(1,-1)

315.0

In [11]:
np.arctan2(1,-1)*180/np.pi
#np.arctan2(y,x)*180/np.pi
def arctan(x,y):
    if np.arctan2(y,x)*180/np.pi >= 0:
        return np.arctan2(y,x)*180/np.pi
    else :
        return np.arctan2(y,x)*180/np.pi + 360

In [4]:
import numpy as np

In [4]:
#x1y2-x2y1

0